# Importation des librairies

In [1]:
import pandas as pd
import numpy as np
import requests as rq
from io import BytesIO
import json
from pyarrow.lib import ArrowInvalid
import pymysql
import psycopg2

## Extraction depuis un api

In [3]:
url_json = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&month=2009-01&outputsize=full&apikey=demo'
url_csv = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=demo&datatype=csv'

In [4]:
response_json = rq.get(url_json)
response_csv = rq.get(url_csv)
print(response_json)

<Response [200]>


In [5]:
response_json.json()

{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume',
  '2. Symbol': 'IBM',
  '3. Last Refreshed': '2009-01-30 19:55:00',
  '4. Interval': '5min',
  '5. Output Size': 'Full size',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (5min)': {'2009-01-30 19:55:00': {'1. open': '50.6335',
   '2. high': '50.6335',
   '3. low': '50.6335',
   '4. close': '50.6335',
   '5. volume': '209'},
  '2009-01-30 17:40:00': {'1. open': '50.6612',
   '2. high': '50.6612',
   '3. low': '50.6612',
   '4. close': '50.6612',
   '5. volume': '84726'},
  '2009-01-30 17:15:00': {'1. open': '50.9626',
   '2. high': '50.9626',
   '3. low': '50.9626',
   '4. close': '50.9626',
   '5. volume': '2651'},
  '2009-01-30 17:10:00': {'1. open': '50.6612',
   '2. high': '50.6612',
   '3. low': '50.6612',
   '4. close': '50.6612',
   '5. volume': '359'},
  '2009-01-30 17:05:00': {'1. open': '50.4677',
   '2. high': '50.4732',
   '3. low': '50.4677',
   '4. close': '50.4732',
   '5. volum

In [6]:
response_csv.text

'timestamp,open,high,low,close,volume\r\n2024-11-27 18:50:00,226.9200,226.9200,226.9200,226.9200,25\r\n2024-11-27 18:40:00,227.2750,227.2750,227.2750,227.2750,4\r\n2024-11-27 18:35:00,227.0100,227.0100,227.0100,227.0100,5\r\n2024-11-27 18:30:00,226.9200,226.9200,226.9000,226.9200,510861\r\n2024-11-27 18:25:00,226.7200,226.7200,226.7200,226.7200,15\r\n2024-11-27 18:20:00,226.7200,226.7200,226.7200,226.7200,1\r\n2024-11-27 18:15:00,226.8500,227.3800,226.5500,227.3800,14\r\n2024-11-27 18:10:00,226.9500,227.1000,226.9500,227.1000,3\r\n2024-11-27 18:05:00,226.5500,226.5500,226.5500,226.5500,1\r\n2024-11-27 18:00:00,226.9500,226.9500,226.9500,226.9500,1\r\n2024-11-27 17:55:00,226.5500,227.0100,226.5500,227.0100,31\r\n2024-11-27 17:50:00,227.3700,227.3700,227.3700,227.3700,1\r\n2024-11-27 17:30:00,226.5500,227.0300,226.5500,227.0300,2\r\n2024-11-27 17:25:00,227.0300,227.0300,226.7725,227.0000,46\r\n2024-11-27 17:15:00,227.3900,227.3900,227.3900,227.3900,50\r\n2024-11-27 17:10:00,227.0000,227.

In [32]:
VALID_FORMATS = ['json', 'csv', 'parquet']
def extract_from_api(url, data_format='json'):

    if data_format not in VALID_FORMATS:
        raise ValueError(f"Format de données invalide. Les formats valides sont : {', '.join(VALID_FORMATS)}")
    # Effectuer la requête GET
    
    response = rq.get(url)
    
    # Vérifier si la réponse est valide
    if response.status_code != 200:
        raise Exception(f"URL incorrecte. Code de statut : {response.status_code}")

    # Traiter selon le format de données
    if data_format == 'json':
        try:
            return response.json()
        except json.JSONDecodeError as e:
            print(f"Erreur : {e} \nFormat de données incorrect")
    
    elif data_format == 'csv':
        return response.text  # CSV sous forme de texte brut
    
    elif data_format == 'parquet':
        # Lire le fichier Parquet directement depuis la mémoire
        try:
            # Lire le fichier Parquet
            return pd.read_parquet(BytesIO(response.content))
        except ArrowInvalid as e:
            print(f"Erreur : {e} \nFormat de données incorrect")

    else:
        raise ValueError(f"Format de données non pris en charge : {data_format}")


In [33]:
url= 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=demo&datatype=csv'
extract_from_api(url=url, data_format='parquet')

Erreur : Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file. 
Format de données incorrect


## Extraction depuis une base de données transactionnelle

In [2]:

def extract_from__rdbms(db_type = 'mysql', connection_info=None, schema=None):
    
    try:
        if db_type == "postgresql":
            conn = psycopg2.connect(
                user=connection_info['user'],
                password=connection_info['password'],
                host=connection_info['host'],
                port=connection_info['port'],
                database=connection_info['database']  
            )
        
        elif db_type == "mysql":
            conn = pymysql.connect(
                user=connection_info['user'],
                password=connection_info['password'],
                host=connection_info['host'],
                port=connection_info['port'],
                database=connection_info['database']  
            )
        
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM {connection_info['table_name']}")
        df = pd.DataFrame(cursor.fetchall(), columns=cursor.description)
        conn.close()
        return df
    
    except Exception as e:
        print(f"Error extracting data from {connection_info['table_name']}: {e}")
        return None

In [4]:
#Test avec Mysql


#Schema="Public"
connection_string = {
    "user": "avnadmin",
    "password": "AVNS_1SIOWzKtEqea945vlFX",
    "host": "mysql-1c4fbc9b-christiansakandelsi-fc70.f.aivencloud.com", 
    "port": 21394,
    "database": "devlivrina_recette",
    "table_name": "model_has_roles"
    }

In [6]:
extract_from__rdbms(db_type="mysql", connection_info=connection_string).head()

,role_id,model_type,model_id
,3,253,8
,NaN,NaN,NaN
,10,764,20
,10,764,20
,0,0,0
,False,False,False
0,1,App\User,1
1,2,App\User,4
2,2,App\User,35
3,2,App\User,68


In [7]:
connection_string = {
    "user": "devlivrina",
    "password": "gKeIMAkn7,@M",
    "host": "dev.livrina.com", 
    "port": 2083,
    "database": "devlivrina_recette-3.5",
    "table_name": "model_has_roles"
    }

In [9]:
extract_from__rdbms(db_type="mysql", connection_info=connection_string)

Error extracting data from model_has_roles: Packet sequence number wrong - got 80 expected 0
